In [1]:
import time
import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

In [28]:
# 自定义函数 -----------------------------------------------------------------------------------------------------
def name_to_str(obj, namespace):
    return [name for name in namespace if namespace[name] is obj]

def find_index(index, pred_type):
    equal = np.empty(0)
    for i in np.arange(0, pred_type.size):
        count = index.count(pred_type[i])
        if count == 0:
            equal = np.append(equal, "False")
        else:
            equal = np.append(equal, "True")
    return pd.DataFrame({"equal":equal})

def result(test_x, test_y):
    # 预测
    pred = model.predict(test_x)
    truth_type = test_y.argmax(axis = 1)
    pred_type = pred.argmax(axis = 1)
    
    # test准确性
    single_accuracy = round(model.evaluate(test_x, test_y)[1], 3)
    print("\n★--- Single_accuracy:", single_accuracy)
    print("\n------------------------------------------------------------------")
    
    # 获取对应索引
    name = name_to_str(pred_x,globals())[0]
    if name == "x_test_normal": class_index = normal_index; total_index = normal_index
    if name == "x_test_speeding": class_index = speeding_index; total_index = aberrant_index
    if name == "x_test_cheating": class_index = cheating_index; total_index = aberrant_index
    if name == "x_test_r_guessing": class_index = r_guessing_index; total_index = aberrant_index
    # print(class_index, total_index)
    
    # class准确性
    class_df = find_index(class_index, pred_type)
    class_accuracy = round(class_df.value_counts()["True"] / class_df.size, 3)
    
    print("\n★--- Class_accuracy:", class_accuracy)
    print("\n------------------------------------------------------------------")
    
    # total准确性
    total_df = find_index(total_index, pred_type)
    total_accuracy = round(total_df.value_counts()["True"] / total_df.size, 3)
    print("\n★--- Total_accuracy:", total_accuracy)
    print("\n------------------------------------------------------------------")
    
    return single_accuracy, class_accuracy, total_accuracy

In [32]:
# 生成保存结果的数组 -----------------
normal_single_accuracy = np.empty(0)
speeding_single_accuracy = np.empty(0)
cheating_single_accuracy = np.empty(0)
r_guessing_single_accuracy = np.empty(0)

normal_class_accuracy = np.empty(0)
speeding_class_accuracy = np.empty(0)
cheating_class_accuracy = np.empty(0)
r_guessing_class_accuracy = np.empty(0)

normal_total_accuracy = np.empty(0)
speeding_total_accuracy = np.empty(0)
cheating_total_accuracy = np.empty(0)
r_guessing_total_accuracy = np.empty(0)

# 重复50次 (1:51) -----------------------------
print(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime()))
for rs in np.arange(1, 2):
    
    ########## 数据参数 ##########
    start_name = "Q_6_20_high_0.2"
    seed = str(rs)
    hasRT = "F"
    ##############################
    print("--------------------- running " + seed + " ---------------------")
    
    # 读入数据 ---------------------
    train_x_path = "./DATA2/" + seed + "s_" + start_name + "_train_x_" + hasRT + ".csv"
    train_y_path = "./DATA2/"  + seed + "s_" + start_name + "_train_y.csv"

    test_normal_x_path = "./DATA2/"  + seed + "s_" + start_name + "_test_normal_x_" + hasRT + ".csv"
    test_normal_y_path = "./DATA2/"  + seed + "s_" + start_name + "_test_normal_y.csv"
    test_cheating_x_path = "./DATA2/"  + seed + "s_" + start_name + "_test_cheating_x_" + hasRT + ".csv"
    test_cheating_y_path = "./DATA2/"  + seed + "s_" + start_name + "_test_cheating_y.csv"
    test_speeding_x_path = "./DATA2/"  + seed + "s_" + start_name + "_test_speeding_x_" + hasRT + ".csv"
    test_speeding_y_path = "./DATA2/"  + seed + "s_" + start_name + "_test_speeding_y.csv"
    test_r_guessing_x_path = "./DATA2/"  + seed + "s_" + start_name + "_test_r_guessing_x_" + hasRT + ".csv"
    test_r_guessing_y_path = "./DATA2/"  + seed + "s_" + start_name + "_test_r_guessing_y.csv"

    x_train = np.array(pd.read_csv(train_x_path).drop("Unnamed: 0", axis=1))
    y_train = np.array(pd.read_csv(train_y_path).drop("Unnamed: 0", axis=1))

    x_test_normal = np.array(pd.read_csv(test_normal_x_path).drop("Unnamed: 0", axis=1))
    y_test_normal = np.array(pd.read_csv(test_normal_y_path).drop("Unnamed: 0", axis=1))
    x_test_cheating = np.array(pd.read_csv(test_cheating_x_path).drop("Unnamed: 0", axis=1))
    y_test_cheating = np.array(pd.read_csv(test_cheating_y_path).drop("Unnamed: 0", axis=1))
    x_test_speeding = np.array(pd.read_csv(test_speeding_x_path).drop("Unnamed: 0", axis=1))
    y_test_speeding = np.array(pd.read_csv(test_speeding_y_path).drop("Unnamed: 0", axis=1))
    x_test_r_guessing = np.array(pd.read_csv(test_r_guessing_x_path).drop("Unnamed: 0", axis=1))
    y_test_r_guessing = np.array(pd.read_csv(test_r_guessing_y_path).drop("Unnamed: 0", axis=1))

    # 获取异常类索引 ---------------
    def get_same_element_index(ob_list, word):
        return [i for (i, v) in enumerate(ob_list) if v == word]

    normal_list = pd.read_csv(train_y_path).drop("Unnamed: 0", axis=1).columns.str.contains('normal').tolist()
    speeding_list = pd.read_csv(train_y_path).drop("Unnamed: 0", axis=1).columns.str.contains('speeding').tolist()
    cheating_list = pd.read_csv(train_y_path).drop("Unnamed: 0", axis=1).columns.str.contains('cheating').tolist()
    r_guessing_list = pd.read_csv(train_y_path).drop("Unnamed: 0", axis=1).columns.str.contains('r_guessing').tolist()

    normal_index = get_same_element_index(normal_list, True)
    aberrant_index = get_same_element_index(normal_list, False)
    speeding_index = get_same_element_index(speeding_list, True)
    cheating_index = get_same_element_index(cheating_list, True)
    r_guessing_index = get_same_element_index(r_guessing_list, True)
    

    # 训练集——测试集划分（5-5） ---
    x_train,x_val,y_train,y_val = train_test_split(x_train,y_train,train_size=0.5,random_state=666)
    print("x_train.shape:", x_train.shape)
    print("y_train.shape:", y_train.shape)
    
    ########## 模型参数 ##########
    layer_one = x_train.shape[1] - 1
    batch_size = int(x_train.shape[0] / 1000)
    learning_rate=0.0001
    epochs = 100
    ##############################
    
    # 构建模型 -----------------------
    model = Sequential()
    model.add(Dense(units = layer_one, input_dim = x_train.shape[1], activation = "relu"))
    model.add(Dense(units = y_train.shape[1], activation = "softmax"))
    
    adam = Adam(learning_rate = learning_rate)
    
    model.compile(optimizer = adam, loss = "categorical_crossentropy", metrics = ['accuracy'])
    
    # 训练模型 ------------------------
    H = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_val, y_val), workers=20, use_multiprocessing=True)
    
    # normal结果 ----------------------
    pred_x = x_test_normal
    pred_y = y_test_normal
    print(name_to_str(pred_x,globals()))
    single_accuracy, class_accuracy, total_accuracy = result(pred_x, pred_y)
    normal_single_accuracy = np.append(normal_single_accuracy, single_accuracy)
    normal_class_accuracy = np.append(normal_class_accuracy, class_accuracy)
    normal_total_accuracy = np.append(normal_total_accuracy, total_accuracy)
    
    # speeding结果 --------------------
    pred_x = x_test_speeding
    pred_y = y_test_speeding
    print(name_to_str(pred_x,globals()))
    single_accuracy, class_accuracy, total_accuracy = result(pred_x, pred_y)
    speeding_single_accuracy = np.append(speeding_single_accuracy, single_accuracy)
    speeding_class_accuracy = np.append(speeding_class_accuracy, class_accuracy)
    speeding_total_accuracy = np.append(speeding_total_accuracy, total_accuracy)

    # cheating结果 ---------------------
    pred_x = x_test_cheating
    pred_y = y_test_cheating
    print(name_to_str(pred_x,globals()))
    single_accuracy, class_accuracy, total_accuracy = result(pred_x, pred_y)
    cheating_single_accuracy = np.append(cheating_single_accuracy, single_accuracy)
    cheating_class_accuracy = np.append(cheating_class_accuracy, class_accuracy)
    cheating_total_accuracy = np.append(cheating_total_accuracy, total_accuracy)

    # r_guessing结果 -------------------
    pred_x = x_test_r_guessing
    pred_y = y_test_r_guessing
    print(name_to_str(pred_x,globals()))
    single_accuracy, class_accuracy, total_accuracy = result(pred_x, pred_y)
    r_guessing_single_accuracy = np.append(r_guessing_single_accuracy, single_accuracy)
    r_guessing_class_accuracy = np.append(r_guessing_class_accuracy, class_accuracy)
    r_guessing_total_accuracy = np.append(r_guessing_total_accuracy, total_accuracy)
    
print(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime()))

2022/11/14 22:59:05
--------------------- running 1 ---------------------
x_train.shape: (92500, 20)
y_train.shape: (92500, 256)
Epoch 1/100
1006/1006 [==============================] - 3s 2ms/step - loss: 5.3643 - accuracy: 0.0143 - val_loss: 5.0947 - val_accuracy: 0.0296
Epoch 2/100
1006/1006 [==============================] - 2s 2ms/step - loss: 4.7955 - accuracy: 0.0606 - val_loss: 4.5236 - val_accuracy: 0.1098
Epoch 3/100
1006/1006 [==============================] - 2s 2ms/step - loss: 4.2746 - accuracy: 0.1638 - val_loss: 4.0481 - val_accuracy: 0.2300
Epoch 4/100
1006/1006 [==============================] - 2s 2ms/step - loss: 3.8273 - accuracy: 0.2774 - val_loss: 3.6293 - val_accuracy: 0.3247
Epoch 5/100
1006/1006 [==============================] - 2s 2ms/step - loss: 3.4334 - accuracy: 0.3600 - val_loss: 3.2622 - val_accuracy: 0.3882
Epoch 6/100
1006/1006 [==============================] - 2s 2ms/step - loss: 3.0927 - accuracy: 0.4184 - val_loss: 2.9485 - val_accuracy: 0.4414
E

1006/1006 [==============================] - 2s 2ms/step - loss: 1.3407 - accuracy: 0.6071 - val_loss: 1.3593 - val_accuracy: 0.6069
Epoch 57/100
1006/1006 [==============================] - 2s 2ms/step - loss: 1.3390 - accuracy: 0.6073 - val_loss: 1.3568 - val_accuracy: 0.6073
Epoch 58/100
1006/1006 [==============================] - 2s 2ms/step - loss: 1.3372 - accuracy: 0.6084 - val_loss: 1.3552 - val_accuracy: 0.6076
Epoch 59/100
1006/1006 [==============================] - 2s 2ms/step - loss: 1.3355 - accuracy: 0.6082 - val_loss: 1.3539 - val_accuracy: 0.6084
Epoch 60/100
1006/1006 [==============================] - 2s 2ms/step - loss: 1.3339 - accuracy: 0.6078 - val_loss: 1.3531 - val_accuracy: 0.6089
Epoch 61/100
1006/1006 [==============================] - 2s 2ms/step - loss: 1.3325 - accuracy: 0.6088 - val_loss: 1.3512 - val_accuracy: 0.6085
Epoch 62/100
1006/1006 [==============================] - 2s 2ms/step - loss: 1.3307 - accuracy: 0.6088 - val_loss: 1.3492 - val_accuracy

In [33]:
# 结果整理
res_dataFrame = pd.DataFrame({"normal_single_accuracy":normal_single_accuracy,
                              "speeding_single_accuracy":speeding_single_accuracy,
                              "cheating_single_accuracy":cheating_single_accuracy,
                              "r_guessing_single_accuracy":r_guessing_single_accuracy,
                              "normal_class_accuracy":normal_class_accuracy,
                              "speeding_class_accuracy":speeding_class_accuracy,
                              "cheating_class_accuracy":cheating_class_accuracy,
                              "r_guessing_class_accuracy":r_guessing_class_accuracy,
                              "normal_total_accuracy":normal_total_accuracy,
                              "speeding_total_accuracy":speeding_total_accuracy,
                              "cheating_total_accuracy":cheating_total_accuracy,
                              "r_guessing_total_accuracy":r_guessing_total_accuracy})
res_dataFrame

,normal_single_accuracy,speeding_single_accuracy,cheating_single_accuracy,r_guessing_single_accuracy,normal_class_accuracy,speeding_class_accuracy,cheating_class_accuracy,r_guessing_class_accuracy,normal_total_accuracy,speeding_total_accuracy,cheating_total_accuracy,r_guessing_total_accuracy
0,0.636,0.486,0.707,0.0,0.851,0.63,0.977,0.01,0.851,0.681,0.997,0.624


In [34]:
# 导出
res_dataFrame.to_csv(path_or_buf = "./res2/" + start_name + "_" + hasRT + ".csv", index=False)